In [57]:
import pandas as pdx
import sqlite3
import sqlalchemy as sa
from sqlalchemy.types import *
from sklearn.preprocessing import LabelEncoder

In [58]:
engine = sa.create_engine('sqlite:////home/roman/BI/python/BI_2020-2021_Python/SQL_homework/database.db')

In [59]:
connection = engine.connect()

# Data preprocessing

Raw data is stored in **raw_data.csv**. Fields are separated with **@**. Because of irregularity of web pages that were parsed, some lines might be completely broken. For the sake of simplicity I have saved only those lines containing all fields without missing values. Most of the popular titles are in, so that is not a big problem. Also data is raw, so it needs some preprocessing, mainly type conversions.

In [60]:
# It is seems like '@' was not the best separator, because it is present in nearly 20 titles
# "Bad lines" are lines containing @ in fields, they are ignored
data = pd.read_csv("raw_data.csv", sep="@", error_bad_lines=False, warn_bad_lines=False, na_values="None").dropna()
data.id = data.id.astype(int)                                      # Convert id to integer
data.score = data.score.astype(float)                              # Convert score to float
data.rating_count = data.rating_count.astype(int)                  # Convert rating_count to integer
data.rank = data["rank"].astype(int)                               # Convert rank to integer
data.popularity = data.popularity.astype(int)                      # Convert popularity to integer
data = data.drop(columns=["favorites"])                            # "favorites" is dropped due to parsing error
data.year = data.year.astype(int)                                  # Convert year to integer
data.episodes = data.episodes.where(data.episodes != "Unknown", 0)   # Convert "Unknown" number of episodes to 0 in order to maintain 1 type
data.genres = data.genres.apply(lambda x: x.split(","))            # Split genres into two columns with coarse and all genres
data["coarse_genre"] = data.genres.apply(lambda x: x[0])
data["all_genres"] = data.genres.apply(lambda x: " ".join(x))

coarse_genre_encoder = LabelEncoder().fit(data.coarse_genre)
all_genres_encoder = LabelEncoder().fit(data.all_genres)

data.coarse_genre = coarse_genre_encoder.transform(data.coarse_genre)
data.all_genres = all_genres_encoder.transform(data.all_genres)

data.studio = data.studio.where(lambda x: x != "add some")         # Ignore parsing error in "studio" field

# Database creation

Because it was very hard to distinguish any logically separated entities, final result may look a bit artifitial.

In [61]:
metadata = sa.schema.MetaData(bind=connection)   # Metadata object is used to bind Tables and Connection

In [62]:
anime_table = sa.Table("anime", metadata, sa.Column("id", INT(), sa.ForeignKey("descriptions.id"),
                                                    sa.ForeignKey("ranks.id"), primary_key=True, autoincrement=True),
                        sa.Column("title_english", VARCHAR(100)),
                        sa.Column("title_japanese", VARCHAR(100)))

description_table = sa.Table("descriptions", metadata, sa.Column("id", INT(), primary_key=True, autoincrement=True),
                                                       sa.Column("synopsys", TEXT(), nullable=True),
                                                       sa.Column("type", VARCHAR(10), nullable=True),
                                                       sa.Column("episodes", INT(), nullable=True),
                                                       sa.Column("rating", VARCHAR(10), nullable=True),
                                                       sa.Column("duration", VARCHAR(20), nullable=True),
                                                       sa.Column("season", VARCHAR(10), nullable=True),
                                                       sa.Column("year", INT(), nullable=True),
                                                       sa.Column("studio", VARCHAR(50), nullable=True),
                                                       sa.Column("coarse_genre", INT(),
                                                                 sa.ForeignKey("coarse_genres.id"),
                                                                 nullable=True),
                                                       sa.Column("all_genres", INT(),
                                                                 sa.ForeignKey("all_genres.id"),
                                                                 nullable=True))

ranks_table = sa.Table("ranks", metadata, sa.Column("id", INT(), sa.ForeignKey("anime.id"), primary_key=True, autoincrement=True),
                                          sa.Column("score", DECIMAL(4, 2)),
                                          sa.Column("rating_count", INT(), nullable=True),
                                          sa.Column("rank", INT(), nullable=True),
                                          sa.Column("popularity", INT(), nullable=True))

coarse_genre_table = sa.Table("coarse_genres", metadata, sa.Column("id", INT(), primary_key=True, autoincrement=True),
                                                         sa.Column("coarse_genre", VARCHAR(20), nullable=True))

all_genres_table = sa.Table("all_genres", metadata, sa.Column("id", INT(), primary_key=True, autoincrement=True),
                                                    sa.Column("all_genres", TEXT(), nullable=True))

In [63]:
# Tables creation
anime_table.create(checkfirst=True)
description_table.create(checkfirst=True)
ranks_table.create(checkfirst=True)
coarse_genre_table.create(checkfirst=True)
all_genres_table.create(checkfirst=True)

Fillings of different tables are separated in order to minimize possible error impact

In [64]:
with connection.begin(): # Create "anime" table 
    for _, row in data.loc[:, ["id", "title_english", "title_japanese"]].iterrows():
        connection.execute(anime_table.insert(), row.to_dict())

In [65]:
with connection.begin(): # Create "ranks" table 
    for _, row in data.loc[:, ["id", 'score', 'rating_count', 'rank', 'popularity']].iterrows():
        connection.execute(ranks_table.insert(), row.to_dict())

In [66]:
with connection.begin(): # Create "descriptions" table 
    for _, row in data.loc[:, ["id", "synopsys", "type", "episodes", "rating", "duration", "season", "year", "studio", "coarse_genre", "all_genres"]].iterrows():
        connection.execute(description_table.insert(), row.to_dict())

In [67]:
with connection.begin(): # Create "coarse_genres" table 
    for id_, genre in enumerate(coarse_genre_encoder.classes_):
        connection.execute(coarse_genre_table.insert(), {"id": id_, "coarse_genre": genre})

In [68]:
with connection.begin(): # Create "all_genres" table 
    for id_, genre in enumerate(all_genres_encoder.classes_):
        connection.execute(all_genres_table.insert(), {"id": id_, "all_genres": genre})

In [55]:
# Removes all tables (for testing)
connection.execute("DROP TABLE anime")
connection.execute("DROP TABLE ranks")
connection.execute("DROP TABLE descriptions")
connection.execute("DROP TABLE coarse_genres")
connection.execute("DROP TABLE all_genres")

In [56]:
connection.close()